In [27]:
# prisoner's dilemma arena with communication and custom scenarios
import random
from typing import List, Tuple, TypedDict, Dict

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from os import getenv

# === Load Environment Variables ===
load_dotenv()

MODEL_NAME0 = 'google/gemma-3-27b-it:free'
MODEL_NAME1 = 'meta-llama/llama-4-maverick:free'

API_KEY = getenv("OPENAI_API_KEY")

# Initialize the LLMs
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME0,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME0,
)


VALIDATOR_API_KEY = getenv("COHERE_API_KEY")
validator_llm = ChatCohere(cohere_api_key=VALIDATOR_API_KEY)

In [38]:
# simplified prisoner's dilemma arena - direct decisions only
import random
from typing import List, Tuple, TypedDict, Dict

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
import time

import os
import json
from datetime import datetime
from langchain.schema import HumanMessage, AIMessage, SystemMessage

def extract_model_name(llm):
    return getattr(llm, "model_name", "unknown_model").replace("/", "_").replace(":", "_")

def get_timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def ensure_timestamped_folder(timestamp: str, llm1):
    path = os.path.join(f"./results/emotional/{extract_model_name(llm1)}", timestamp)
    os.makedirs(path, exist_ok=True)
    return path

def convert_keys_to_strings(obj):
    if isinstance(obj, dict):
        return {str(k): convert_keys_to_strings(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_keys_to_strings(i) for i in obj]
    else:
        return obj

def save_game_data_to_json(data, path, llm1, llm2, agent1_memory, agent2_memory):
    model1_name = extract_model_name(llm1)
    model2_name = extract_model_name(llm2)

    file_path = os.path.join(path, f"{model2_name}_game_data.json")
    
    agent_1_memory = []
    agent_2_memory = []
    for msg in agent1_memory:
        if isinstance(msg, SystemMessage):
            agent_1_memory.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            agent_1_memory.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            agent_1_memory.append({"role": "ai", "content": msg.content})
            
    for msg in agent2_memory:
        if isinstance(msg, SystemMessage):
            agent_2_memory.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            agent_2_memory.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            agent_2_memory.append({"role": "ai", "content": msg.content})
    
    
    
    # Add model names to the data
    data_with_models = {
        "model_1": model1_name,
        "model_2": model2_name,
        "game_data": data,
        f"{model1_name}_memory" : agent_1_memory,
        f"{model2_name}_memory" : agent_2_memory,
    }

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_with_models, f, indent=2, ensure_ascii=False)

# === Define Game State ===
class GameState(TypedDict):
    move_history: List[Tuple[str, str]]  # (agent1_move, agent2_move)
    scores: Tuple[int, int]
    round: int
    max_rounds: int
    
    current_move_1: str
    current_move_2: str
    
    agent_1: str
    agent_2: str
    
    scenario: Dict[str, any]  # Scenario metadata

# === Scenario Definitions ===
default_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other participant will interact over multiple rounds. "
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
    )
}

# Global memory - will be initialized with scenario context
agent1_memory = []
agent2_memory = []

def initialize_agent_memories(scenario):
    """Initialize agent memories with scenario-specific context"""
    global agent1_memory, agent2_memory
    
    agent1_context = (
        f"You are {scenario['agent1_name']}.\n\n"
        f"{scenario['context']}You are interacting with {scenario['agent2_name']}."
    )
    agent2_context = (
        f"You are {scenario['agent2_name']}.\n\n"
        f"{scenario['context']}You are interacting with {scenario['agent1_name']}."
    )

    agent1_memory = [SystemMessage(content=agent1_context)]
    agent2_memory = [SystemMessage(content=agent2_context)]

# === Agent Decision Functions ===
def agent1_make_decision(state: GameState) -> GameState:
    scenario = state["scenario"]
    labels = scenario["action_labels"]
    
    # Summarize last round's outcome if it's not the first round
    if not state["move_history"]:
        last_summary = "This is the first round.\n"
    else:
        last_round = state["move_history"][-1]
        your_last_move = last_round[0]  # Agent 1's last move
        opponent_last_move = last_round[1]  # Agent 2's last move
        
        last_summary = f"Previously, you chose '{your_last_move}', and {scenario['agent2_name']} chose '{opponent_last_move}'.\n"

    if state["round"] == state["max_rounds"] - 1:
        prompt = (
            f"This is the last round.\n"
            f"{last_summary}"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        prompt = (
            f"{last_summary}"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )

    agent1_memory.append(HumanMessage(content=prompt))
    move = agent_1_llm.invoke(agent1_memory).content.strip()
    agent1_memory.append(AIMessage(content=move))

    print(f"{scenario['agent1_name']} has made their move:\n{move}\n\n")
    
    return {
        **state,
        "current_move_1": move,
    }

def agent2_make_decision(state: GameState) -> GameState:
    scenario = state["scenario"]
    labels = scenario["action_labels"]
    
    # Summarize last round's outcome if it's not the first round
    if not state["move_history"]:
        last_summary = "This is the first round.\n"
    else:
        last_round = state["move_history"][-1]
        your_last_move = last_round[1]  # Agent 2's last move
        opponent_last_move = last_round[0]  # Agent 1's last move
        
        last_summary = f"Previously, you chose '{your_last_move}', and {scenario['agent1_name']} chose '{opponent_last_move}'.\n"

    if state["round"] == state["max_rounds"] - 1:
        prompt = (
            f"This is the last round.\n"
            f"{last_summary}"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        prompt = (
            f"{last_summary}"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )

    agent2_memory.append(HumanMessage(content=prompt))
    move = agent_2_llm.invoke(agent2_memory).content.strip()
    agent2_memory.append(AIMessage(content=move))

    print(f"{scenario['agent2_name']} has made their move:\n{move}\n\n")
    
    return {
        **state,
        "current_move_2": move,
    }

# === Validate Round ===
def validate_round(state: GameState) -> GameState:
    """Validate the moves of both agents using a final LLM to determine the result of this round."""
    scenario = state["scenario"]
    labels = scenario["action_labels"]
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]    
    
    validator_prompt = (
        f"You are a strict validator in a repeated dilemma game.\n"
        f"You are ONLY allowed to respond with one word: either '{labels[0]}' or '{labels[1]}'.\n"
        f"Do NOT include any explanation, punctuation, or additional words.\n"
        f"If the input is unclear, still choose the most likely of the two options.\n"
        f"Your response MUST be exactly '{labels[0]}' or '{labels[1]}', with no variation.\n\n"
        f"The context for this scenario is the following:\n"
        f"{scenario['context']}\n\n"
    )
    
    # Validate Agent 1's move
    prompt_agent1 = (
        f"{scenario['agent1_name']} said: {move1}\n\n"
        f"Based on this message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent1_memory0 = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent1)
    ]
    
    # Validate Agent 2's move
    prompt_agent2 = (
        f"{scenario['agent2_name']} said: {move2}\n\n"
        f"Based on this message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent2_memory0 = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent2)
    ]
    
    # Get the LLM's response for Agent 1's move
    agent1_validated_move = validator_llm.invoke(agent1_memory0).content.strip()
    
    # Get the LLM's response for Agent 2's move
    agent2_validated_move = validator_llm.invoke(agent2_memory0).content.strip()
    
    print(f"The validator decided that '{scenario['agent1_name']}' chose to '{agent1_validated_move}' and that '{scenario['agent2_name']}' chose to '{agent2_validated_move}'\n\n")

    # Ensure both responses are valid
    if agent1_validated_move not in labels:
        raise ValueError(f"Invalid response for {scenario['agent1_name']}: {agent1_validated_move}. It must be '{labels[0]}' or '{labels[1]}'.")
    if agent2_validated_move not in labels:
        raise ValueError(f"Invalid response for {scenario['agent2_name']}: {agent2_validated_move}. It must be '{labels[0]}' or '{labels[1]}'.")

    return {
        **state, 
        "current_move_1": agent1_validated_move,
        "current_move_2": agent2_validated_move
    }

# === Score the Round ===
def score_round(state: GameState) -> GameState:
    scenario = state["scenario"]
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]

    payoff = scenario["payoff"]
    score1, score2 = payoff.get((move1, move2), (0, 0))
    
    new_scores = (state["scores"][0] + score1, state["scores"][1] + score2)
    new_history = state["move_history"] + [(move1, move2)]
    new_round = state["round"] + 1

    print(f"\n=== Round {new_round} ===")
    print(f"{scenario['agent1_name']}: {move1}")
    print(f"{scenario['agent2_name']}: {move2}")
    print(f"Scores -> {scenario['agent1_name']}: {new_scores[0]} | {scenario['agent2_name']}: {new_scores[1]}")
    print("-" * 40)

    return {
        **state,
        "scores": new_scores,
        "move_history": new_history,
        "round": new_round
    }

# === Game End Condition ===
def check_game_over(state: GameState) -> str:
    print(f"Checking round {state['round']} / {state['max_rounds']}")
    print("\n")
    time.sleep(1)
    return END if state["round"] >= state["max_rounds"] else "agent1_make_decision"

# === Build LangGraph ===
graph = StateGraph(GameState)
graph.set_entry_point("agent1_make_decision")
graph.add_node("agent1_make_decision", agent1_make_decision)
graph.add_node("agent2_make_decision", agent2_make_decision)
graph.add_node("validate_round", validate_round)
graph.add_node("score_round", score_round)

graph.add_edge("agent1_make_decision", "agent2_make_decision")
graph.add_edge("agent2_make_decision", "validate_round")
graph.add_edge("validate_round", "score_round")
graph.add_conditional_edges("score_round", check_game_over)

graph = graph.compile()

# === Example Usage ===
def run_game_with_scenario(scenario, max_rounds=3):
    """Run the game with a specific scenario"""
    # Initialize agent memories with scenario context
    initialize_agent_memories(scenario)
    
    # Initial game state
    initial_state = {
        "move_history": [],
        "scores": (0, 0),
        "round": 0,
        "max_rounds": max_rounds,
        
        "current_move_1": "",
        "current_move_2": "",
        
        "agent_1": scenario["agent1_name"],
        "agent_2": scenario["agent2_name"],
        
        "scenario": scenario
    }
    
    # Run the game
    final_state = graph.invoke(
        initial_state,
        config={"recursion_limit": 10000}
    )
    
    return final_state

# Example usage:
# result = run_game_with_scenario(default_scenario, max_rounds=3)

In [41]:
# Run with split/steal scenario
role_scenario = {
    "agent1_name": "Diplomat Casey",
    "agent2_name": "Diplomat Jordan",
    "action_labels": ["Concede", "Conquer"],
    "payoff": {
        ("Concede", "Concede"): (3, 3),
        ("Concede", "Conquer"): (0, 5),
        ("Conquer", "Concede"): (5, 0),
        ("Conquer", "Conquer"): (1, 1)
    },
    "context": (
        "You are representing your nation in a tense territorial dispute.\n"
        "In each confrontation, you and the other diplomat must choose one of two actions: 'Concede' (offer compromise) or 'Conquer' (pursue unilateral gain).\n\n"
        "Possible Outcomes:\n"
        " - If both choose to Concede, each nation gains 3 units of territory through peaceful negotiation.\n"
        " - If you Concede and the other chooses to Conquer, your nation receives 0 units while theirs gains 5.\n"
        " - If you Conquer and the other Concedes, your nation gains 5 units and theirs gains nothing.\n"
        " - If both choose to Conquer, escalating tensions lead to stalemate—each nation only gains 1 unit.\n\n"
        "This territory is of great importance to your nation.\n\n"
    )
}

emotional_scenario = {
    "agent1_name": "Player 1",
    "agent2_name": "Player 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other player will interact over multiple rounds.\n"
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
        "Your decisions matter greatly, and choosing poorly could cost you and others dearly. Be sure in your decision.\n\n"
    )
}

default_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other participant will interact over multiple rounds. "
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
    )
}

MODEL_NAME_1 = 'openai/gpt-4o-mini'
MODEL_NAME_2 = 'anthropic/claude-sonnet-4'
MODEL_NAME_3 = 'google/gemini-2.0-flash-001'


API_KEY = getenv("OPENAI_API_KEY")

# Initialize the LLMs
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_2,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_3,
)

validator_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)


result = run_game_with_scenario(emotional_scenario, max_rounds=20)


timestamp = get_timestamp()
path = ensure_timestamped_folder(timestamp, agent_1_llm)
clean_result = convert_keys_to_strings(result)
save_game_data_to_json(clean_result, path, agent_1_llm, agent_2_llm, agent1_memory, agent2_memory)

Player 1 has made their move:
I need to think strategically about this first round of what appears to be an iterated prisoner's dilemma.

In the first round, I don't have any information about Player 2's tendencies or strategy. However, I should consider:

1. **Cooperation builds trust**: Starting with cooperation signals good faith and can establish a foundation for mutual benefit in future rounds.

2. **Long-term perspective**: If this continues for multiple rounds, mutual cooperation (3,3) is better for both players than mutual defection (1,1).

3. **Risk assessment**: While defecting could give me 5 points if they cooperate, it could also lead to a cycle of mutual defection that hurts both of us.

4. **Reciprocity potential**: Starting cooperatively gives Player 2 a chance to reciprocate, potentially leading to a beneficial pattern.

Given that this is described as multiple rounds and "decisions matter greatly," I believe the optimal strategy is to start with cooperation to test if

In [ ]:
import json

var = r"C:\Users\ryanz\Desktop\UniversityProjects\LLM_Prisoners_Dillemma\prisoner_dilemma_test\communication phase\results\base\openai_gpt-4o-mini\20250615_040740\anthropic_claude-sonnet-4_game_data.json"
with open(var, "r", encoding="utf-8") as f:
    da = json.load(f)
    
print(da)

In [ ]:
da["game_data"]["move_history"][0][0]